In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45103
2,Huelva,Confirmados PDIA 14 días,390
3,Huelva,Tasa PDIA 14 días,"75,99820722177836"
4,Huelva,Confirmados PDIA 7 días,162
5,Huelva,Tasa PDIA 7 dias,"31,568486076738704"
6,Huelva,Total Confirmados,45305
7,Huelva,Curados,42942
8,Huelva,Fallecidos,411


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45103.0


/tmp/ipykernel_3951/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12476.0


/tmp/ipykernel_3951/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3951/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3951/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3951/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4359 personas en los últimos 7 días 

Un positivo PCR cada 1821 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45103.0,162.0,390.0,513170.0,31.568486,75.998207,23.0
Huelva-Costa,26705.0,102.0,268.0,289548.0,35.227320,92.558056,15.0
Condado-Campiña,13989.0,49.0,84.0,156231.0,31.363814,53.766538,6.0
Huelva (capital),12476.0,33.0,79.0,143837.0,22.942636,54.923281,6.0
Moguer,1943.0,11.0,18.0,21867.0,50.304111,82.315818,3.0
Aljaraque,1616.0,5.0,7.0,21474.0,23.283971,32.597560,2.0
Almonte,2268.0,11.0,22.0,24507.0,44.885135,89.770270,2.0
Isla Cristina,3109.0,11.0,39.0,21393.0,51.418688,182.302622,2.0
Lepe,2993.0,14.0,28.0,27880.0,50.215208,100.430416,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Higuera de la Sierra,67.0,2.0,5.0,1291.0,154.918668,387.296669,1.0
Paterna del Campo,205.0,10.0,12.0,3457.0,289.268152,347.121782,0.0
Santa Bárbara de Casa,46.0,2.0,3.0,1043.0,191.754554,287.631831,0.0
Cartaya,1950.0,17.0,56.0,20083.0,84.648708,278.842802,0.0
Cañaveral de León,33.0,1.0,1.0,396.0,252.525253,252.525253,0.0
Aroche,128.0,1.0,7.0,3054.0,32.743942,229.207597,0.0
Ayamonte,2049.0,17.0,46.0,21104.0,80.553450,217.968158,1.0
Isla Cristina,3109.0,11.0,39.0,21393.0,51.418688,182.302622,2.0
Minas de Riotinto,178.0,2.0,6.0,3812.0,52.465897,157.397692,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,128.0,1.0,7.0,3054.0,32.743942,229.207597,0.0,0.142857
Isla Cristina,3109.0,11.0,39.0,21393.0,51.418688,182.302622,2.0,0.282051
Cartaya,1950.0,17.0,56.0,20083.0,84.648708,278.842802,0.0,0.303571
Sierra de Huelva-Andévalo Central,3987.0,11.0,34.0,67391.0,16.322654,50.451841,2.0,0.323529
Minas de Riotinto,178.0,2.0,6.0,3812.0,52.465897,157.397692,0.0,0.333333
Ayamonte,2049.0,17.0,46.0,21104.0,80.553450,217.968158,1.0,0.369565
Huelva-Costa,26705.0,102.0,268.0,289548.0,35.227320,92.558056,15.0,0.380597
Higuera de la Sierra,67.0,2.0,5.0,1291.0,154.918668,387.296669,1.0,0.400000
Punta Umbría,1257.0,2.0,5.0,15355.0,13.025073,32.562683,1.0,0.400000
